In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt"

--2025-08-04 20:45:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2025-08-04 20:45:47 (297 MB/s) - ‘companypolicies.txt’ saved [15660/15660]



In [3]:
# @title Installations
# The %%capture command hides the installation output.
%%capture
!pip install langchain
!pip install lark
!pip install openai
!pip install chromadb
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_community


In [4]:
# @title Imports
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
import os
from google.colab import userdata
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

In [5]:
# @title Docs
loader = TextLoader("companypolicies.txt")
txt_data = loader.load()
txt_data

[Document(metadata={'source': 'companypolicies.txt'}, page_content="1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20) # Use RecursiveCharacterTextSplitter
chunks_txt = text_splitter.split_documents(txt_data)

In [7]:
# @title Environment Setup
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [8]:
# @title Chroma DB and OpenAI Embeddings
vectordb = Chroma.from_documents(chunks_txt, OpenAIEmbeddings())

In [9]:
# @title Simple similarity search
query = "email policy"
retriever = vectordb.as_retriever()
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy is established to guide the responsible and secure use of these essential tools within our organization. We recognize their significance in daily business operations and'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Compliance: Ensure compliance with all relevant laws and regulations regarding internet and email usage, including those related to copyright and data protection.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content="Acceptable Use: Company-provided internet and email services are primarily meant for job-related tasks. Limited personal use is allowed during non-work hours, provided it doesn't interfere with work")]

In [10]:
# @title limit the retrieval results.
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy is established to guide the responsible and secure use of these essential tools within our organization. We recognize their significance in daily business operations and')]

In [11]:
# @title MMR search.

docsretriever = vectordb.as_retriever(search_type="mmr")
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Internet and Email Policy is established to guide the responsible and secure use of these essential tools within our organization. We recognize their significance in daily business operations and')]

In [13]:
# @title Similarity score threshold retrieval
retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='3.\tInternet and Email Policy')]